In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from tqdm.notebook import tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform, pdist
import haversine as hs
from scipy.spatial.distance import cdist
from create_dist_matrix import create_dist_matrix, create_dist_mat_2
from get_avail_act import get_avail_act
from get_best_action import get_best_action
from get_or_tools_sol import or_solution
from get_random_traj import get_random_traj
from train import train_model
import numpy as np


In [2]:
import pandas as pd

data_csv = pd.read_csv("clean_data.csv")
df = pd.DataFrame(data_csv)


In [3]:
df.drop_duplicates(subset=['CourierId','Latitude','Longitude'],keep='first',inplace=True)

df

CourierId   Latitude  Longitude
0          324293  55.605452  12.583987
3          324293  55.631359  12.656990
8          324293  55.638429  12.634550
12         324293        NaN        NaN
15         324293  55.656790  12.604109
...           ...        ...        ...
254923          0  55.569114  12.618886
254924          0  55.568978  12.619223
254925          0  55.568047  12.618471
254926          0  55.570509  12.620062
254927          0  55.569847  12.617192

[67774 rows x 3 columns]

In [4]:
df = df.dropna(axis=0)
index_list = [i for i in range(len(df))]
df = df.set_index([index_list])

df

CourierId   Latitude  Longitude
0         324293  55.605452  12.583987
1         324293  55.631359  12.656990
2         324293  55.638429  12.634550
3         324293  55.656790  12.604109
4         324293  55.657004  12.604765
...          ...        ...        ...
67658          0  55.569114  12.618886
67659          0  55.568978  12.619223
67660          0  55.568047  12.618471
67661          0  55.570509  12.620062
67662          0  55.569847  12.617192

[67663 rows x 3 columns]

In [8]:
alpha = 0.012
gamma = 0.4
def tsp_solver(dist_mat,alpha=0.2,gamma=0.8):
    #alpha is the learning rate
    #gamma is the discount factor
    n_dest = dist_mat.shape[0]
    # Train RL model
    q = train_model(dist_mat, n_train = 2000, gamma = gamma, alpha = alpha)# Get trained transition matrix

    #print(q)

    # Use model to find optimum trajectory
    state = [0]
    distance_travel = 0.
    posible_actions = get_avail_act(state, n_dest)
    while posible_actions: # until all destinations are visited
        action = get_best_action(state[-1], posible_actions, q)
        distance_travel += dist_mat[state[-1], action]
        state.append(action)
        posible_actions = get_avail_act(state, n_dest)

    #Back to warehouse
    action = 0
    distance_travel += dist_mat[state[-1], action]
    state.append(action)

    # Get Best optimization possible
    #print("\nGoogle Results: ")
    best_dist, google_route = or_solution(dist_mat)

    # Get random tour
    random_dist, random_route = get_random_traj(dist_mat)

    #Out RL results
    traj =' -> '.join([str(b) for b in state])
    #print(f"Best trajectory found with RL: \n {traj}" )
    #print(f"Total distance travelled with this traj: {distance_travel}\n")
    slow_pctg = 100*(-1+distance_travel/best_dist)
    random_pctg = 100*(-1+distance_travel/random_dist)
    return slow_pctg, traj, distance_travel, google_route, best_dist
    #print(f"RL solution is {100*(-1+distance_travel/best_dist)}% slower than google's solution")

dist_list = [0] * len(df)
df['Distance'] = dist_list
new_df = df.loc[df.CourierId == 324293]
new_df = new_df.set_index([[i for i in range(len(new_df))]])
dist_matrix = [[0]*len(new_df) for _ in range(len(new_df))]

for j in new_df.index:
    for k in new_df.index:
        loc1=(new_df.Latitude[j], new_df.Longitude[j])
        loc2=(new_df.Latitude[k], new_df.Longitude[k])
        dist_matrix[j][k] = hs.haversine(loc1,loc2)

dist_matrix = np.array(dist_matrix)

slow_pctg, rl_route, rl_dist, google_route, google_dist = tsp_solver(dist_matrix, alpha=alpha, gamma=gamma)
print(rl_route)
print(rl_dist)

0 -> 11 -> 12 -> 10 -> 3 -> 4 -> 2 -> 1 -> 7 -> 15 -> 5 -> 8 -> 16 -> 13 -> 17 -> 6 -> 9 -> 14 -> 0
49.03878441870544


C:\Users\21269\AppData\Local\Temp/ipykernel_16084/726995205.py:38: RuntimeWarning: divide by zero encountered in double_scalars
  slow_pctg = 100*(-1+distance_travel/best_dist)
